# Data Cleaning and Preparation

a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up `80%` or more of an analyst’s time.

In this lesson I discuss tools for **missing data**, **duplicate data**, **string manipulation**,
and some other analytical data transformations. 


In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Interacting with Databases

In a business setting, most data may not be stored in text or Excel files. SQL-based
relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use,
and many alternative databases have become quite popular.

In [47]:
# Load employees data from sqlite database 'hr.db' using a SQL query

from sqlite3 import connect
import os

path = os.getcwd() + "\\hr.db"
with connect(path) as conn:
    data = pd.read_sql("SELECT * FROM employees", conn)
    
data.head()



,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.0,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.0,100.0,9
2,102,Lex,De Haan,lex.de haan@sqltutorial.org,515.123.4569,1993-01-13,5,17000.0,100.0,9
3,103,Alexander,Hunold,alexander.hunold@sqltutorial.org,590.423.4567,1990-01-03,9,9050.0,102.0,6
4,104,Bruce,Ernst,bruce.ernst@sqltutorial.org,590.423.4568,1991-05-21,9,6000.0,103.0,6


In [48]:
# Load employees data from sqlite database 'hr.db' using read table
# https://docs.sqlalchemy.org/en/14/core/engines.html
from sqlalchemy import create_engine

conn = create_engine("sqlite:///hr.db")
data = pd.read_sql_table("employees", conn)
data.head()


,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.0,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.0,100.0,9
2,102,Lex,De Haan,lex.de haan@sqltutorial.org,515.123.4569,1993-01-13,5,17000.0,100.0,9
3,103,Alexander,Hunold,alexander.hunold@sqltutorial.org,590.423.4567,1990-01-03,9,9050.0,102.0,6
4,104,Bruce,Ernst,bruce.ernst@sqltutorial.org,590.423.4568,1991-05-21,9,6000.0,103.0,6


## Handling Missing Data

- All of the descriptive statistics on pandas objects exclude missing data by default.
- For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data.

In [49]:
string_data = pd.Series([None, 'aardvark', 'artichoke', np.nan, 'avocado'])

In [50]:
# check the missing values with isnull() function
string_data.isnull().sum()


2

![](assets/na-methods.png)

### Filtering Out Missing Data

In [51]:
# filter out the missing data (first approach)
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
mask = data.notnull()
data[mask]

0    1.0
2    3.5
4    7.0
dtype: float64

In [52]:
# filter out the missing data (second approach)
data.dropna()


0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop **rows**
or **columns** that are **all** `NA` or only those containing **any** `NAs`.

In [53]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [54]:
# Drop any row containing a missing value
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [55]:
# Drop any row with all values missing
data.dropna(how="all")


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [56]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [57]:
# drop the columns that have all values missing 

data.dropna(axis=1, how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [58]:
df = pd.DataFrame(np.random.randint(0, 10, (7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df


,0,1,2
0,2,NaN,NaN
1,8,NaN,NaN
2,8,NaN,0.0
3,6,NaN,4.0
4,6,6.0,9.0
5,3,3.0,2.0
6,3,3.0,2.0


In [59]:
# drop all rows that have any missing values
df.dropna()

,0,1,2
4,6,6.0,9.0
5,3,3.0,2.0
6,3,3.0,2.0


In [60]:
# drop all rows that have 2 or more missing values
df.dropna(thresh=2)

,0,1,2
2,8,NaN,0.0
3,6,NaN,4.0
4,6,6.0,9.0
5,3,3.0,2.0
6,3,3.0,2.0


### Filling In Missing Data
Rather than filtering out missing data (and potentially discarding other data along
with it), you may want to fill in the “holes” in any number of ways.

For most purposes, the `fillna` method is the workhorse function to use.

In [61]:
# replace all missing data with 0
df.fillna(0)

,0,1,2
0,2,0.0,0.0
1,8,0.0,0.0
2,8,0.0,0.0
3,6,0.0,4.0
4,6,6.0,9.0
5,3,3.0,2.0
6,3,3.0,2.0


Calling `fillna` with a **dict**, you can use a different fill value for each column:


In [62]:
# replace missing values of column 1 by 0.5 and of column 2 by 0

df.fillna({1: 0.5, 2: 0})

,0,1,2
0,2,0.5,0.0
1,8,0.5,0.0
2,8,0.5,0.0
3,6,0.5,4.0
4,6,6.0,9.0
5,3,3.0,2.0
6,3,3.0,2.0


`fillna` returns a **new object**, but you can modify the existing object in-place

In [63]:
# fill the values in-place
df.fillna(0, inplace=True)

In [64]:
df

,0,1,2
0,2,0.0,0.0
1,8,0.0,0.0
2,8,0.0,0.0
3,6,0.0,4.0
4,6,6.0,9.0
5,3,3.0,2.0
6,3,3.0,2.0


In [65]:
df = pd.DataFrame(np.random.randint(0, 10, (6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df


,0,1,2
0,2,5.0,1.0
1,9,6.0,9.0
2,1,NaN,0.0
3,2,NaN,0.0
4,0,NaN,NaN
5,5,NaN,NaN


the parameter **method** is a powerful utility available for `fillna` method

In [66]:
# fill the missing value with the value precedes it
df.fillna(method="ffill")


,0,1,2
0,2,5.0,1.0
1,9,6.0,9.0
2,1,6.0,0.0
3,2,6.0,0.0
4,0,6.0,0.0
5,5,6.0,0.0


In [67]:
# fill the missing value with the value precedes it with maximum of 2 filling
df.fillna(method="ffill", limit=2)



,0,1,2
0,2,5.0,1.0
1,9,6.0,9.0
2,1,6.0,0.0
3,2,6.0,0.0
4,0,NaN,0.0
5,5,NaN,0.0


**check**: fill the missing value with the mean

In [68]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [69]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

![](assets/fillna-args.png)

## Data Transformation
So far in this lesson we’ve been concerned with rearranging data. Filtering, cleaning,
and other transformations are another class of important operations.

### Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an
example:

In [70]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a boolean Series indicating whether each
row is a duplicate (has been observed in a previous row) or not:

In [71]:
# which row(S) is duplicated 
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [72]:
data.duplicated(keep="last")

0    False
1    False
2    False
3    False
4    False
5     True
6    False
dtype: bool

In [74]:
data.duplicated(keep=False)

0    False
1    False
2    False
3    False
4    False
5     True
6     True
dtype: bool

In [75]:
data.duplicated().sum()

1

Relatedly, `drop_duplicates` returns a DataFrame where the duplicated array is
False

In [76]:
# show the rows that are not duplicated
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider **all** of the columns; alternatively, you can
specify any **subset** of them to detect duplicates.

In [77]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [80]:
# drop rows with duplicated values at column k1
# data.duplicated(subset=["k1"]).sum()
data.drop_duplicates(subset="k1")

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one

In [81]:
# drop rows with duplicated values at columns k1 and k2, keeping the last opservation
data.drop_duplicates(subset=["k1", "k2"], keep="last")


,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the val‐
ues in an array, Series, or column in a DataFrame.

In [86]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food
came from.

In [87]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

The `map` method on a Series accepts a **function** or **dict-like** object containing a mapping, 
but here we have a small problem in that **some** of the meats are **capitalized** and
others are not. Thus, we need to convert each value to lowercase using the `str.lower`
Series method

In [84]:
# convert all strings in 'food' column to lower case and assign it to a variable
lower_food = data["food"].str.lower()
lower_food

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [85]:
# add new column 'animal' to the dataframe and get the matching value from the dict 'meat_to_animal'
data["animal"] = lower_food.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [89]:
# approach 2, pass a function that do all the work
def map_food_to_animal(food):
    return meat_to_animal[food.lower()]

data["animal"] = data["food"].map(map_food_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [90]:
# approach 2, pass a function that do all the work
# def map_food_to_animal(food):
#     return meat_to_animal[food.lower()]

data["animal"] = data["food"].map(lambda food: meat_to_animal[food.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value
replacement. As you’ve already seen, `map` can be used to modify a subset of values in
an object but `replace` provides a simpler and more flexible way to do so. 

Let’s consider this Series

In [91]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [93]:
# replace the value of -999 with np.nan
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [94]:
data.map(lambda x: np.nan if x < 0 else x)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value

In [95]:
# replace both the values of -999 and -1000 with np.nan
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes, or a mapping dict 


In [96]:
# replace both the values of -999 and -1000 with np.nan and 0 respectivly
data.replace([-999, -1000], [np.nan, 0])


0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [97]:
# replace both the values of -999 and -1000 with np.nan and 0 respectivly

data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or mapping 
of some form to produce new, differently labeled objects. You can also modify
the axes in-place without creating a new data structure.

In [111]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a map method

In [103]:
# convert the indices to upper case
data.index = data.index.map(lambda x: x[:3].upper())

If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`

In [113]:
# convert the indices and column names to upper case using rename method
# data.rename(index=lambda x: x[:3].upper(), inplace=True)
# data
data.rename(index=str.upper, columns=str.upper)

,ONE,TWO,THREE,FOUR
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [115]:
# rename spcific index and column
data.rename(index={"Ohio": "Riyadh"}, columns={"three": "Olia"})

,one,two,Olia,four
Riyadh,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [117]:
# rename index in-place
data.rename(index={"Ohio": "Riyadh"}, columns={"three": "Olia"}, inplace=True)
data

,one,two,Olia,four
Riyadh,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis.
Suppose you have data about a group of people in a study, and you want to group
them into discrete age buckets

In [135]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use `cut`, a function in pandas

In [136]:
# divide the ages to 4 classes using the bins
bins = [18, 25, 35, 60, 100]
age_groups = pd.cut(ages, bins)
age_groups

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special **Categorical object**. The output you see
describes the bins computed by `pandas.cut`. 

You can treat it like an **array of strings** indicating the bin name; 

internally it contains a **categories array** specifying the distinct category names along with a labeling for the ages data in the **codes attribute**:

In [137]:
# print the ages codes
age_groups.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [138]:
# print the distinct categories
age_groups.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [139]:
# print the number of occurrances of each category 
age_groups.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

**Note** that `cats.value_counts()` are the bin counts for the result of `pandas.cut`.
Consistent with mathematical notation for intervals, a **parenthesis** means that the side
is **open**, while the **square bracket** means it is **closed (inclusive)**. 

You can change which side is closed by passing `right=False`:

In [140]:
# categorize the ages with right open intervals
age_groups = pd.cut(ages, bins, right=False)

age_groups

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can also pass your own bin names by passing a list or array to the labels option:

In [141]:
# categorize the ages with the labels below
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
age_groups = pd.cut(ages, bins, labels=group_names)
age_groups

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.

In [142]:
data = np.random.randint(0, 20, 12)
data

array([15, 13, 13, 12, 15,  2, 13, 10, 10,  4, 17,  9])

In [143]:
# divide the data to 4 groups
cut = pd.cut(data, 4, precision=0)
cut

[(13.0, 17.0], (10.0, 13.0], (10.0, 13.0], (10.0, 13.0], (13.0, 17.0], ..., (10.0, 13.0], (10.0, 13.0], (2.0, 6.0], (13.0, 17.0], (6.0, 10.0]]
Length: 12
Categories (4, interval[float64]): [(2.0, 6.0] < (6.0, 10.0] < (10.0, 13.0] < (13.0, 17.0]]

In [144]:
# count the number of values in each group
cut.value_counts()

(2.0, 6.0]      2
(6.0, 10.0]     1
(10.0, 13.0]    6
(13.0, 17.0]    3
dtype: int64

A closely related function, `qcut`, bins the data based on sample quantiles. 

Depending on the distribution of the data, using `cut` will not usually result in each bin having the
same number of data points. 

Since `qcut` uses sample quantiles instead, by definition you will obtain **roughly equal-size** bins:

In [145]:
data = np.random.randint(0, 100, 1000)  # Normally distributed
# Cut into quartiles and count the values
cuts = pd.qcut(data, 4, precision=2)
cuts.value_counts()

(-0.01, 24.0]    262
(24.0, 50.5]     238
(50.5, 75.0]     252
(75.0, 99.0]     248
dtype: int64

## Independent Practice
using the `coffee-preferences.csv` data set
- show the duplicated rows
- show the names of customers who rated all coffee
- show the names of the coffees that are rated by all customers
- which coffee got the highest rate in average
- replace the missing value by zero
- convert names to upper case
- convert the rate to a percentage 